# Multi-Free Field Analysis Example

This example shows how to run OpenSeesMP in DesignSafe from a jupyter notebook.

A set of four 1D profiles is analyzed using OpenSeesMP. 

<img src = "multi-freeField.png"  height="400" width="400" align = "center">

# Setup Tapis and start OpenSeesMP job

### Setup job description

In [4]:
!pip install jsonschema --user 

In [1]:
!pip install --upgrade pip --user -q
!pip install tapipy -q
from tapipy.tapis import Tapis
import json

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-server 2.25.0 requires jsonschema>=4.18.0, but you have jsonschema 4.17.3 which is incompatible.
jupyter-events 0.8.0 requires jsonschema[format-nongpl]>=4.18.0, but you have jsonschema 4.17.3 which is incompatible.


In [2]:
# Method using getpass --- PREFERRED
# =================================

from getpass import getpass
# from tapipy.tapis import Tapis

t = Tapis(
  base_url = "https://designsafe.tapis.io",
  username = getpass('input username: '),
  password = getpass('input password: '))
t.get_tokens()

input username:  ········
input password:  ········


In [5]:
# Import libraries
import os
import uuid
from datetime import date
from DS_GenFunctionsV3 import * #General python functions useful for DesignSafe
# ---------------------------------------------------------------------------------
# Identify folder with input file in DesignSafe
cur_dir        = os.getcwd()
input_uri      = DS_GetDir(cur_dir, t)
print(input_uri)

tapis://designsafe.storage.default/kks32/OpenSees-T3


In [6]:
# ---------------------------------------------------------------------------------
# Define job name
jobname = "opensees-MP-multiMotion-Jupyter"

# ---------------------------------------------------------------------------------
# Select tapis-app
app_id          = "opensees-mp-s3"
app_version     = "3.7.0"

# ---------------------------------------------------------------------------------
# Define file to run
input_filename = "Main_multiMotion.tcl"

# ---------------------------------------------------------------------------------
# Define control tapis-app variables
control_storage_id       = "designsafe.storage.default"
control_execSystem       = "stampede3" #"frontera", "stampede3-simcenter"
control_allocation       = "-A DesignSafe-SimCenter"  # MUST USE YOUR OWN ALLOCATION !!
control_exec_Dir         = "DS_input"  # Folder with files including input_filename
control_nodeCount        = 1
control_corespernode     = 16
control_memoryMB         = 1000
control_maxMinutes       = 60

if control_execSystem == "frontera":
    control_batchQueue = "development"  #"normal"
elif control_execSystem == "stampede3" :
    control_batchQueue = "skx-dev"      # "skx"
elif control_execSystem == "stampede3-simcenter":
    control_batchQueue = "simcenter"
else:
    print("wrong execSystem")

In [9]:
# -------------------------------------------------------------------------------
# Define inputs
fileInputs = [{"name": "Input Directory", "sourceUrl": f"{input_uri}/{control_exec_Dir}"}]

# -------------------------------------------------------------------------------
# Define parameters
parameterSet = {"appArgs":[]}
parameterSet["appArgs"].append({"name": "Main Script", "arg": input_filename})
parameterSet["schedulerOptions"] = [{"arg": control_allocation}]

In [10]:
# -------------------------------------------------------------------------------
# Define job description
app = t.apps.getApp(appId=app_id, appVersion=app_version)
job_description = {}
job_description["name"] = jobname
job_description["appId"] = app_id
job_description["appVersion"] = app_version

job_description["execSystemLogicalQueue"] = control_batchQueue
job_description["execSystemId"] = control_execSystem
job_description["execSystemExecDir"] = "${JobWorkingDir}"

job_description["archiveSystemId"] = control_storage_id
job_description["archiveSystemDir"] = "${EffectiveUserId}/tapis-jobs-archive/${JobCreateDate}/${JobUUID}"

job_description["maxMinutes"] = control_maxMinutes
job_description["nodeCount"] = control_nodeCount
job_description["coresPerNode"] = control_corespernode

job_description["fileInputs"] = fileInputs
job_description["parameterSet"] = parameterSet
print(json.dumps(job_description, indent=2))

{
  "name": "opensees-MP-multiMotion-Jupyter",
  "appId": "opensees-mp-s3",
  "appVersion": "3.7.0",
  "execSystemLogicalQueue": "skx-dev",
  "execSystemId": "stampede3",
  "execSystemExecDir": "${JobWorkingDir}",
  "archiveSystemId": "designsafe.storage.default",
  "archiveSystemDir": "${EffectiveUserId}/tapis-jobs-archive/${JobCreateDate}/${JobUUID}",
  "maxMinutes": 60,
  "nodeCount": 1,
  "coresPerNode": 16,
  "fileInputs": [
    {
      "name": "Input Directory",
      "sourceUrl": "tapis://designsafe.storage.default/kks32/OpenSees-T3/DS_input"
    }
  ],
  "parameterSet": {
    "appArgs": [
      {
        "name": "Main Script",
        "arg": "Main_multiMotion.tcl"
      }
    ],
    "schedulerOptions": [
      {
        "arg": "-A DesignSafe-SimCenter"
      }
    ]
  }
}


### Run job

In [11]:
# Submit job
job = t.jobs.submitJob(**job_description)
jobUuid=job.uuid

print(" Job launched. Status provided below")
print(" Can also check in DedignSafe portal under - Workspace > Tools & Application > Job Status")

BadRequestError: message: No more authentication methods available

In [ ]:
#Check status
status = DS_GetStatus(t, jobUuid, tlapse=1)

# Check Runtime
DS_GetRuntime(t, jobUuid)

# Postprocess Results

### Identify job, archived location and user

In [ ]:
jobinfo = t.jobs.getJob(jobUuid=job.uuid)
jobinfo.archiveSystemDir
user = jobinfo.createdby

### Go to archived folder

In [ ]:
os.chdir(jobinfo.archiveSystemDir.replace(user,'/home/jupyter/MyData'))
# os.chdir(control_exec_Dir)
os.chdir("inputDirectory")

### Import python libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### Plot acceleration response spectra

Plot acceleration response spectra on log-linear scale

In [ ]:
from plotAcc import plot_acc
plot_acc()